# Computation Graph and Expressions

In [1]:
import dynet as dy

dy.renew_cg()

v1 = dy.inputVector([1,2,3,4])

v2 = dy.inputVector([5,6,7,8])

v3 = v1 + v2

v4 = v3 * 2

v5 = v1 + 1

v6 = dy.concatenate([v1,v2,v3,v5])

print(v6)

print(v6.npvalue())

expression 5/1
[ 1.  2.  3.  4.  5.  6.  7.  8.  6.  8. 10. 12.  2.  3.  4.  5.]


In [2]:
model = dy.Model()

pW = model.add_parameters((20,4))
pb = model.add_parameters(20)

dy.renew_cg()

x = dy.inputVector([1,2,3,4])
W = dy.parameter(pW)
b = dy.parameter(pb)

y = W * x + b

print(y)

print(y.npvalue())

The dy.parameter(...) call is now DEPRECATED.
        There is no longer need to explicitly add parameters to the computation graph.
        Any used parameter will be added automatically.
expression 4/2
[-1.88650417  2.45489597  1.46270537  0.0768325  -1.59835815  1.6260407
  1.10095823  2.36807108 -0.11708686  1.72553992  0.80943757  2.03034449
  1.37247717 -1.62450778  0.28080553 -0.69154924 -0.37141031 -1.03671265
  0.0253149  -3.24785519]


In [3]:
model = dy.Model()

trainer = dy.SimpleSGDTrainer(model)

p_v = model.add_parameters(10)

EPOCHS = 10

for epoch in range (EPOCHS):
    dy.renew_cg()
    
    v = dy.parameter(p_v)
    v2 = dy.dot_product(v,v)
    v2.forward()
    
    v2.backward()
    
    trainer.update()
    
print(v2.npvalue())

[0.01109748]


In [4]:
# XOR problem

import dynet as dy
import random

data = [([0,1],0),
        ([1,0],0),
        ([0,0],1),
        ([1,1],1)]

model = dy.Model()
pU = model.add_parameters((4,2))
pb = model.add_parameters(4)
pv = model.add_parameters(4)

trainer = dy.SimpleSGDTrainer(model)




EPOCHS = 1000

for epoch in range (EPOCHS):
    epoch_loss = 0.0
    for x,y in data:
        dy.renew_cg()
        U = dy.parameter(pU)
        b = dy.parameter(pb)
        v = dy.parameter(pv)
        x = dy.inputVector(x)
        
        # prediction
        
        yhat = dy.logistic(dy.dot_product(v,dy.tanh(U * x + b)))
        
        # calculate loss
        if y == 0:
            loss = -dy.log(1 - yhat)
        else:
            loss = -dy.log(yhat)
            
        epoch_loss += loss.scalar_value()
        
        loss.backward()
        trainer.update()
        
    print("Epoch %d. loss = %f" % (epoch, epoch_loss))
    
inputData = [0,0]
i = dy.inputVector(inputData)
yhat = dy.logistic(dy.dot_product(v,dy.tanh(U * x + b)))

print(yhat.value())
            
        
        
        
    

Epoch 0. loss = 3.375566
Epoch 1. loss = 3.282142
Epoch 2. loss = 3.209222
Epoch 3. loss = 3.151414
Epoch 4. loss = 3.104814
Epoch 5. loss = 3.066624
Epoch 6. loss = 3.034836
Epoch 7. loss = 3.008006
Epoch 8. loss = 2.985084
Epoch 9. loss = 2.965297
Epoch 10. loss = 2.948063
Epoch 11. loss = 2.932942
Epoch 12. loss = 2.919591
Epoch 13. loss = 2.907736
Epoch 14. loss = 2.897161
Epoch 15. loss = 2.887685
Epoch 16. loss = 2.879157
Epoch 17. loss = 2.871454
Epoch 18. loss = 2.864466
Epoch 19. loss = 2.858103
Epoch 20. loss = 2.852283
Epoch 21. loss = 2.846937
Epoch 22. loss = 2.842006
Epoch 23. loss = 2.837435
Epoch 24. loss = 2.833178
Epoch 25. loss = 2.829195
Epoch 26. loss = 2.825448
Epoch 27. loss = 2.821907
Epoch 28. loss = 2.818544
Epoch 29. loss = 2.815334
Epoch 30. loss = 2.812254
Epoch 31. loss = 2.809286
Epoch 32. loss = 2.806412
Epoch 33. loss = 2.803617
Epoch 34. loss = 2.800886
Epoch 35. loss = 2.798208
Epoch 36. loss = 2.795572
Epoch 37. loss = 2.792966
Epoch 38. loss = 2.790

Epoch 987. loss = 0.034711
Epoch 988. loss = 0.034667
Epoch 989. loss = 0.034624
Epoch 990. loss = 0.034580
Epoch 991. loss = 0.034537
Epoch 992. loss = 0.034493
Epoch 993. loss = 0.034450
Epoch 994. loss = 0.034407
Epoch 995. loss = 0.034364
Epoch 996. loss = 0.034321
Epoch 997. loss = 0.034279
Epoch 998. loss = 0.034236
Epoch 999. loss = 0.034193
0.9890434741973877


In [2]:
# Deep Averaging Network (Look-up table)

import dynet as dy
import random
import pandas as pd
import numpy as np
import csv

model = dy.Model()

l2i = {}
i2l = {}

V = 1000
HID = 20
EDIM = 50
NOUT = 5


def predict_labels(doc):
    x = encode_doc(doc)
    h= layer1(x)
    y = layer2(h)
    
    return dy.softmax(y)


def layer1(x):
    W = dy.parameter(pW1)
    b = dy.parameter(pb1)
    
    return dy.tanh(W * x + b)

def layer2(x):
    W = dy.parameter(pW2)
    b = dy.parameter(pb2)
    
    return dy.tanh(W * x + b)

def encode_doc(doc):
    doc = [w2i[w] for w in doc]
    embs = [E[idx] for idx in doc]
    
    return dy.esum(embs)

def do_loss(probs, label):
    label = l2i[label]
    
    return -dy.log(dy.pick(probs,label))

def classify(doc):
    dy.renew_cg()
    probs = predict_labels(doc)
    
    vals = probs.npvalue()
    
    return i2l[np.argmax(vals)]

def read_dataset(fname):
    print('Reading dataset...')
    df = pd.read_csv(fname)

    tags_index = {
        "sci-fi": 1,
        "action": 2,
        "comedy": 3,
        "fantasy": 4,
        "animation": 5,
        "romance": 6,
    }
    
    index_tags = {
        1 : "sci-fi",
        2 : "action",
        3 : "comedy",
        4 : "fantasy",
        5 : "animation",
        6 : "romance",
    }
   
    data = []
    with open(fname) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                line_count += 1
            else:
                data.append([row[2],tags_index[row[3].strip("\n")]])
                line_count += 1

    return data, tags_index,index_tags

def read_glove():
    print('Loading word vectors...')
    word2vec = {}
    embedding = []
    embeds = []
    word2idx = {}
    with open('glovo/glove.6B.50d.txt', encoding='utf-8') as f:
      for line in f:
        values = line.split()
        word = values[0]
        word2idx[word] = len(embeds)
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
        embeds.append(vec)
    return np.array(embeds),word2idx
    
    

    
    
data, l2i, i2l = read_dataset("tagged_plots_movielens.csv")
embedding , w2i = read_glove()

pW1 = model.add_parameters((HID, EDIM))
pb1 = model.add_parameters(HID)
pW2 = model.add_parameters((NOUT, HID))
pb2 = model.add_parameters(NOUT)
E = model.add_lookup_parameters((len(w2i), EDIM),init =  embedding )

# for (doc, label) in data:
#     dy.renew_cg()
#     probs = predict_labels(doc)
    
#     loss = do_loss(probs,label)
#     loss.forward()
#     loss.backward()
#     trainer.update()
    

Reading dataset...
Loading word vectors...


In [3]:
for (doc,label) in data:
    print(doc,"--",label)
    break

A little boy named Andy loves to be in his room, playing with his toys, especially his doll named "Woody". But, what do the toys do when Andy is not with them, they come to life. Woody believes that he has life (as a toy) good. However, he must worry about Andy's family moving, and what Woody does not know is about Andy's birthday party. Woody does not realize that Andy's mother gave him an action figure known as Buzz Lightyear, who does not believe that he is a toy, and quickly becomes Andy's new favorite toy. Woody, who is now consumed with jealousy, tries to get rid of Buzz. Then, both Woody and Buzz are now lost. They must find a way to get back to Andy before he moves without them, but they will have to pass through a ruthless toy killer, Sid Phillips. -- 5
